Yield curve forecasting
=================
This jupyter notebook demonstrates a handful of forecasting methods in yield curve modelling. The script loads the historical data from the US threasury and lets the user to choose the window for training of the forecasting algorithm as well as the forecasting step.

The notebook is a suplement to the report Ayliffe and Rubin [1] which grew as an extension of Kelly Ayliffe's semester project at Ecole Polytechnique Fédéral de Lausanne, Switzerland, written under the supervision of Tomas Rubin.

Contacts:
- Ayliffe Kelly, kellyayliffe@gmail.com
- Tomas Rubin, tomas.rubin@gmail.com

Yield curve data
------------
We consider the problem of forecasting the yields of the [US treasury](https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/textview.aspx?data=yield) yield curve rates. The institution publishes every business day yield curve quotes with maturities: 	1 Mo,	2 Mo,	3 Mo,	6 Mo,	1 Yr,	2 Yr,	3 Yr,	5 Yr,	7 Yr,	10 Yr,	20 Yr,	30 Yr.

Bellow is a screenshot of the data for April 1 --- April 22, 2020.

| Date     | 1 Mo | 2 Mo | 3 Mo | 6 Mo | 1 Yr | 2 Yr | 3 Yr | 5 Yr | 7 Yr | 10 Yr | 20 Yr | 30 Yr |
|----------|------|------|------|------|------|------|------|------|------|-------|-------|-------|
| 04/01/20 | 0.03 | 0.07 | 0.09 | 0.14 | 0.16 | 0.23 | 0.28 | 0.37 | 0.51 | 0.62  | 1.04  | 1.27  |
| 04/02/20 | 0.09 | 0.10 | 0.09 | 0.15 | 0.14 | 0.23 | 0.29 | 0.39 | 0.53 | 0.63  | 1.04  | 1.26  |
| 04/03/20 | 0.09 | 0.11 | 0.10 | 0.15 | 0.15 | 0.23 | 0.30 | 0.39 | 0.52 | 0.62  | 1.05  | 1.24  |
| 04/06/20 | 0.09 | 0.13 | 0.15 | 0.17 | 0.20 | 0.27 | 0.35 | 0.44 | 0.58 | 0.67  | 1.08  | 1.27  |
| 04/07/20 | 0.10 | 0.13 | 0.14 | 0.20 | 0.20 | 0.28 | 0.36 | 0.48 | 0.64 | 0.75  | 1.13  | 1.32  |
| 04/08/20 | 0.14 | 0.17 | 0.22 | 0.24 | 0.23 | 0.27 | 0.34 | 0.47 | 0.65 | 0.77  | 1.18  | 1.37  |
| 04/09/20 | 0.20 | 0.27 | 0.25 | 0.24 | 0.25 | 0.23 | 0.29 | 0.41 | 0.60 | 0.73  | 1.15  | 1.35  |
| 04/13/20 | 0.17 | 0.29 | 0.26 | 0.27 | 0.27 | 0.25 | 0.31 | 0.44 | 0.63 | 0.76  | 1.19  | 1.39  |
| 04/14/20 | 0.17 | 0.19 | 0.20 | 0.24 | 0.25 | 0.23 | 0.29 | 0.42 | 0.61 | 0.76  | 1.19  | 1.41  |
| 04/15/20 | 0.14 | 0.15 | 0.14 | 0.19 | 0.19 | 0.20 | 0.24 | 0.34 | 0.49 | 0.63  | 1.06  | 1.27  |
| 04/16/20 | 0.14 | 0.15 | 0.14 | 0.18 | 0.17 | 0.20 | 0.25 | 0.35 | 0.50 | 0.61  | 1.01  | 1.21  |
| 04/17/20 | 0.12 | 0.12 | 0.12 | 0.16 | 0.16 | 0.20 | 0.26 | 0.36 | 0.53 | 0.65  | 1.08  | 1.27  |
| 04/20/20 | 0.10 | 0.10 | 0.12 | 0.15 | 0.15 | 0.20 | 0.26 | 0.35 | 0.51 | 0.63  | 1.04  | 1.23  |
| 04/21/20 | 0.08 | 0.08 | 0.11 | 0.14 | 0.17 | 0.20 | 0.24 | 0.34 | 0.48 | 0.58  | 0.98  | 1.17  |
| 04/22/20 | 0.09 | 0.09 | 0.12 | 0.14 | 0.16 | 0.22 | 0.26 | 0.37 | 0.52 | 0.63  | 1.03  | 1.22  |

Forecasting methods
--------
In this notebook we consider the following methods:
1. **[DNS 2-step]** The dynamic Nelson-Siegel model as considered by Diebold and Li [2], estimation by the *two-step method*. The method firstly fits the level, slope and curvature factors at each cross-section and produces vector autoregression forecasts on the factors level.
2. **[DNS 1-step]** The dynamic Nelson-Siegel model as considered by Diebold et al. [3], estimation by the *one-step method*. This methods views the setting from 1. as a state space model and estimates the latent vector autoregression using the expectation maximisation (EM) algorithm and predicts the yield curve using the Kalman filter.
3. **[DNS 1-step (explosivity correction)]** Again, the dynamic Nelson-Siegel model as considered by Diebold et al. [3] estimated by the *one-step method*. However, if the state space vector autoregression is estimated to be an explosive process, the dynamics is replaced to be random walk in the state space, i.e. the vector autoregression with identity transition matrix.
4. **[RW]** Random walk. This method simply replicates the last known yield as the forecast. Due to very low signal-to-noise ratio in yield curve data and the behaviour of the yield curve dynamics, the random walk is actually a very difficult method to beat, as already noticed by Diebold and Li [2].
5. **[AR]** Scalar autoregression. This model fits a scalar autoregressive model of order 1 for each maturity individually.
6. **[VAR]** Vector autoregression. This model fits a vector autoregressive model of order 1 for the vector of 11 maturities.

The above listed methods are presented in more detail in the report Ayliffe and Rubin [1].


References
-----
[1] Ayliffe, Kelly, and Rubin, Tomas (2020). "A Quantitative Comparison of Yield Curve Models in the MINT Economies." *EPFL Infoscience.* URL: https://infoscience.epfl.ch/record/279314

[2] Diebold, Francis X., and Canlin Li. "Forecasting the term structure of government bond yields." Journal of econometrics 130.2 (2006): 337-364.

[3] Diebold, Francis X., Glenn D. Rudebusch, and S. Boragan Aruoba. "The macroeconomy and the yield curve: a dynamic latent factor approach." Journal of econometrics 131.1-2 (2006): 309-338.

In [ ]:
# USER INPUT HERE
import datetime
start_date = datetime.datetime(2012, 11,  1) # starting date for the training window, first available date 2011/11/01
end_date   = datetime.datetime(2015, 12, 31) # end date for the training windows, last available date 2019/10/31
forecast_step = 365 # how many *day* to forecast forward

Import packages and data
---------------

In [ ]:
# IMPORT PACKAGES
import numpy as np
import numpy.matlib
import math
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime  
from datetime import timedelta  
from mpl_toolkits import mplot3d

# import our custom functions
from yield_curve_functions import DNS_OLS, DNS_formula, forecast_RW_fct, params_VAR
from yield_curve_functions import forecast_DNS_VAR, forecast_DNS_VAR, forecast_VAR, forecast_AR, forecast_DNS_KF
from yield_curve_functions import forecast_DNS_VAR_yw, forecast_DNS_VAR_yw, forecast_VAR_yw, forecast_AR_yw, forecast_DNS_KF_explosivcor

In [ ]:
# IMPORT DATA
y_df = pd.read_excel('US_daily.xlsx', columns = ['dates',1/12,3/12,6/12,1,2,3,5,7,10,20,30], index='dates');
y = y_df.to_numpy()
matu = np.array([[1/12,3/12,6/12,1,2,3,5,7,10,20,30]])

# subset of the data based on user's input
y = y[ (y[:,0] >= start_date) & (y[:,0] <= end_date) ,:]

In [ ]:
# IMPORT DATA
y_full_df = pd.read_excel('US_daily.xlsx', columns = ['dates',1/12,3/12,6/12,1,2,3,5,7,10,20,30], index='dates');
y_full = y_full_df.to_numpy()
matu = np.array([[1/12,3/12,6/12,1,2,3,5,7,10,20,30]])

dates = y_full[:,0]

current=1 #this variable keeps track of the dates in the original dataset that have already been added. It is a row index in the original table.
currentDate = np.datetime64(dates[0]) # this variable keeps track of all dates that need to be added.

# The following two tables will be concatenated horizontally to create the full, new dataset
CompleteTable = np.array([y_full[0,1:]]) #Table with added yields (has copied lines where extra dates have been added)
CompleteDates = np.array([[currentDate]], dtype='datetime64') #Will be the full dates column

AddDay = np.timedelta64(1,'D')

cdnp = np.array([[currentDate]],dtype='datetime64') #single entry array. Used to have a compatible format (np.array) for adding the dates to CompleteDates.

while current<y_full.shape[0]:
    currentDate = currentDate + AddDay
    cdnp[0][0] = currentDate
    CompleteDates = np.hstack((CompleteDates,cdnp))
    dateInTable = np.datetime64(dates[current])
    
    if dateInTable != currentDate:
        CompleteTable = np.vstack((CompleteTable,CompleteTable[-1])) #copies last available line into the table
        
    if dateInTable == currentDate:
        CompleteTable = np.vstack((CompleteTable,y_full[current,1:])) #adds yield curve corresponding to currentDate
        current = current + 1

#Updating to full table
y_full = np.hstack((CompleteDates.transpose(), CompleteTable))

# subset of the data based on user's input
y = y_full[ (y_full[:,0] >= start_date) & (y_full[:,0] <= end_date) ,:]

# find the forecast date
forecast_date = end_date + timedelta(days=forecast_step)
y_forecast = y_full[ y_full[:,0] == forecast_date, 1: ]
if y_forecast.size == 0:
    print("Error: the given forecast step fall outside of the dataset")

# seperating dates and yields
dates = np.array([y[:,0]])
y = np.delete(y,0,1)
y = np.array(y,dtype = float)

Surface plot of the current data window
-----------
The yield curves are usually plotted as a three dimensional plot where on the x-axis one plots the observed maturities, on the y-axis rolls over the considered cross-sections

In [ ]:
plot_x = matu
#plot_y = [x.year for x in dates[0,:]]

def plot_start_date(start_date):
    decimal_start = start_date.year
    index_in_year = start_date.day
    month = start_date.month
    if(month>=2):
        index_in_year+=31
    if(month>=3):
        index_in_year+=28
    if(month>=4):
        index_in_year+=31
    if(month>=5):
        index_in_year+=30
    if(month>=6):
        index_in_year+=31
    if(month>=7):
        index_in_year+=30
    if(month>=8):
        index_in_year+=31
    if(month>=9):
        index_in_year+=31
    if(month>=10):
        index_in_year+=30
    if(month>=11):
        index_in_year+=31
    if(month>=12):
        index_in_year+=30
    
    decimal_start+= index_in_year / 365
    return decimal_start

plot_y = (np.arange((dates[0,:]).shape[0])/365) + plot_start_date(start_date)

plot_x, plot_y = np.meshgrid(plot_x, plot_y)
plot_z = y

fig = plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')

ax.plot_surface(plot_x, plot_y, plot_z,cmap='viridis', edgecolor='none')
ax.set_title('US treasury yields in the selected window')
ax.set_ylabel('')
plt.show()

Produce the forecasts of the yield curve
--------------------

In [ ]:
# OLS fitting of the coefficients
our_lambda = 0.496 #result of commented computation
ts = DNS_OLS(y,matu,our_lambda)
tsf = pd.DataFrame(ts) #some functions require the data in Pandas instead of Numpy

data = y
dataf = pd.DataFrame(y)

p = matu.shape[1] #p = nb maturities
nb_dates = y.shape[0]

In [ ]:
# PRODUCE FORECASTS
forecast_RW = forecast_RW_fct(data,forecast_step) #Random walk
step_two_VAR = forecast_DNS_VAR(tsf,forecast_step) #Two Step DNS with VAR(1)
step_two_VAR_yw = forecast_DNS_VAR_yw(ts,forecast_step) #Two Step DNS with VAR(1), method of moments estimator
forecast_VARn = forecast_VAR(dataf,forecast_step)
forecast_VARn_yw = forecast_VAR_yw(data,forecast_step)
forecast_ARn = forecast_AR(data,forecast_step)
forecast_ARn_yw = forecast_AR_yw(data,forecast_step)

#We choose to initialise the EM algorithm with VAR fitted parameters, because they are likely to make it converge to the global maximum.
state_init = ts[0]
params_init = params_VAR(tsf)
offset_init = params_init[0,:]
transition_init = np.array(params_init[1:,:]).transpose()

forecast_KF = forecast_DNS_KF(dataf,matu,our_lambda,state_init,offset_init,transition_init,forecast_step) #One Step DNS with VAR(1).
forecast_KF_explosivcor = forecast_DNS_KF_explosivcor(dataf,matu,our_lambda,state_init,offset_init,transition_init,forecast_step) #One Step DNS with VAR(1).

Visualise the forecasts
-------

In [ ]:
tau_grid = np.linspace(start=0.001, stop=30, num=100)

fig = plt.figure(figsize=(15,8))
#plt.plot( tau_grid, DNS_formula( tau_grid, step_two_VAR[-1,:] , our_lambda), 'b--' )
plt.plot( tau_grid, DNS_formula( tau_grid, step_two_VAR_yw[-1,:] , our_lambda), 'b-' )
plt.plot( tau_grid, DNS_formula( tau_grid, forecast_KF[-1,:], our_lambda ), 'r--'  )
plt.plot( tau_grid, DNS_formula( tau_grid, forecast_KF_explosivcor[-1,:], our_lambda ), 'r-'  )
#plt.plot( matu.flatten(), forecast_ARn[-1,:], color='grey', marker='o', ls=':' )
plt.plot( matu.flatten(), forecast_ARn_yw[-1,:], color='grey', marker='o', ls='--' )
#plt.plot( matu.flatten(), forecast_VARn[-1,:], color='orange', marker='o', ls=':' )
plt.plot( matu.flatten(), forecast_VARn_yw[-1,:], color='orange', marker='o', ls='--' )
plt.plot( matu.flatten(), forecast_RW[-1,:], color='green', marker='o', ls='-.' )
plt.plot( matu.flatten(), y_forecast.flatten() ,'k-o')
plt.title('US treasury yields in selected window')
plt.legend(['DNS 2-step','DNS 1-step','DNS 1-step (explosivity correction)','AR','VAR','RW','truth'])
plt.xlabel('maturity [years]')
plt.ylabel('yield [percents]')
plt.show() 